In [1]:
from IPython.display import display, Latex
from functools import wraps
from orderedset import OrderedSet
import sympy
from sympy.core.symbol import symbols, Symbol
from sympy.solvers.solveset import nonlinsolve
from sympy import solveset
from sympy import Eq, sqrt, pi, arg, Abs

from sympy.utilities.lambdify import lambdify
from sympy.printing import latex, pprint
from sympy import Symbol, simplify
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, EngFormatter

from math import log10
from PyLTSpice import LTSpice_RawRead
from sympy.solvers import solve
import sys
import logging
from ipywidgets import FloatSlider, interact, interactive
from converter6 import * #works acceptably with converter3

Found Numpy. Will be used for storing data


In [2]:
def make_symbols():
    global R1, R2, C,  s, Vc, Vg, VL,   f,  Ts, w, j
    R1 = variable('R_1', 'ohms', None, real=True, positive=True)
    R2 = variable('R_2', 'ohms', None, real=True, positive=True)
    C = variable('C', 'F', None, real=True, positive=True)
    f = variable('f', 'Hz', real=True, positive=True)
    w = variable('omega', 'Hz', real=True, positive=True)
    #w = 2*pi*f
    Vc = variable('V_c', 'V', real=True)
    Vg = variable('V_g', 'V', real= True)
    VL = variable('V_L', 'V', real= True)
    j = sqrt(-1)

    return
make_symbols()


In [33]:
#exp1 = w - 2*pi*f
exp2 = j*w*C*R2/(1+(R1+R2)*j*w*C)
exp3 = Abs(exp2) - 3.3e-3
exp4 = arg(exp2) - 12*pi/180
#myprint2(exp1)
myprint2(exp4)
myprint2(exp3)
R2_ = 100

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [34]:
exp3.subs({R2:R2_})

100*C*omega/sqrt(C**2*R_1**2*omega**2 + 200*C**2*R_1*omega**2 + 10000*C**2*omega**2 + 1) - 0.0033

In [35]:
exp = Abs(R1+1/(j*w*C)).subs({w:2*pi*85e3}) - 1e3*R2_ #R2 chosen as 1k 1ohms 
exp

sqrt(R_1**2 + 3.46020761245675e-11/(pi**2*C**2)) - 100000.0

In [36]:
exp4 = arg(exp2).subs({w:2*pi*85e3, R2:R2_}) - 12*pi/180
exp4

arg(I/(170000.0*I*pi*C*(R_1 + 100) + 1)) - pi/15

In [37]:
ans = nonlinsolve([exp, exp4], [R1, C])
ans

{(R_1, -5.88235294117647e-11*sqrt(-1/(1.0e-5*R_1 - 1.0))/(pi*sqrt(1.0e-5*R_1 + 1.0))), (R_1, 5.88235294117647e-11*sqrt(-1/(1.0e-5*R_1 - 1.0))/(pi*sqrt(1.0e-5*R_1 + 1.0)))}

In [17]:
ans1 = ans.args[0]
ans1

(R_1, -5.88235294117647e-12*sqrt(-1/(1.0e-6*R_1 - 1.0))/(pi*sqrt(1.0e-6*R_1 + 1.0)))

In [20]:
ans1[1].subs({R1:100e3})

-5.9119871485836e-12/pi

In [38]:
ans2 = ans.args[1]
ans2

(R_1, 5.88235294117647e-11*sqrt(-1/(1.0e-5*R_1 - 1.0))/(pi*sqrt(1.0e-5*R_1 + 1.0)))

In [42]:
ans2[1].subs({R1:80e3}).evalf(3)

3.12e-11

In [44]:
Abs(1/(j*2*pi*85e3*1.155e-6)).evalf(3)

1.62

In [ ]:
c= converter()
#c.add_equation(exp1)
#c.add_equation(exp3)
#c.add_equation(exp4)
#c.equations.add(exp1)
c.equations.add(exp3)
c.equations.add(exp4)
c.solver(given={R2:1e3, w:2*pi*85e3}, want=R1)